In [11]:
import pandas as pd
import plotly.express as px

In [8]:
file_path = 'Data/airports.csv'

In [9]:
df = pd.read_csv(file_path)

In [12]:
df.head(5)

,faa,name,lat,lon,alt,tz,dst,tzone
0,AAF,Apalachicola Regional Airport,29.727501,-85.027496,20,-5.0,A,America/New_York
1,AAP,Andrau Airpark,29.722500,-95.588303,79,-6.0,A,America/Chicago
2,ABE,Lehigh Valley International Airport,40.652100,-75.440804,393,-5.0,A,America/New_York
3,ABI,Abilene Regional Airport,32.411301,-99.681900,1791,-6.0,A,America/Chicago
4,ABL,Ambler Airport,67.106300,-157.856989,334,-9.0,A,America/Anchorage


## Exploration Plotly Express scatter_geo

In [16]:
px.scatter_geo(df, lat="lat", lon="lon",  color="alt", projection="natural earth")
